In [ ]:
# Installing the required dependencies
!pip install datasets
!pip install nltk
!pip install bs4
!pip install gensim
!pip install nlpaug

In [ ]:
# Loading in the required libraries
from datasets import load_dataset
import random
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import tqdm as tqdm
from bs4 import BeautifulSoup
from gensim.utils import simple_preprocess
from gensim.models.word2vec import Word2Vec
import numpy as np
from imblearn.datasets import make_imbalance
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks  
import nlpaug.augmenter.word as naw
import nlpaug.flow as nafc
from nlpaug.util import Action
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import f1_score
import statistics
from numpy import vstack

In [ ]:
# Importing the IMDb dataset from Hugging Face and creating the different splits
imdb_dataset = load_dataset("imdb")
imdb_train_text = imdb_dataset["train"]["text"]
imdb_train_y = np.array(imdb_dataset["train"]["label"])
imdb_test_text = imdb_dataset["test"]["text"]
imdb_test_y = np.array(imdb_dataset["test"]["label"])
semi_size = random.sample(range(0, 50000), 10000)
imdb_unlabeled = imdb_dataset["unsupervised"].select(semi_size)["text"]
imdb_unlabeled_y = np.array(imdb_dataset["unsupervised"].select(semi_size)["label"])

In [ ]:
# Simulating the moderate imbalanced data scenario
num_majority = int((len(imdb_train_text))/2)
num_minority = int((25/75)*num_majority)
imdb_text_array = np.asarray(imdb_train_text).reshape(-1, 1)
imdb_text_moderate, imdb_y_moderate = make_imbalance(imdb_text_array, imdb_train_y, sampling_strategy = {0: num_minority, 1: num_majority}, random_state = 123)
imdb_text_moderate = imdb_text_moderate.tolist()
imdb_text_moderate = [''.join(ele) for ele in imdb_text_moderate]

In [ ]:
# Preprocessing our training data
stopwords_list = stopwords.words("english")
processed_reviews_train = []
for review in tqdm.tqdm(imdb_train_text):
  raw = BeautifulSoup(review)
  html_remove = raw.get_text()
  temp = simple_preprocess(html_remove)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_train.append(temp_update)

# Preprocessing our testing data
processed_reviews_test = []
for review in tqdm.tqdm(imdb_test_text):
  raw = BeautifulSoup(review)
  html_remove = raw.get_text()
  temp = simple_preprocess(html_remove)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_test.append(temp_update)

# Preprocessing our imbalanced data set
processed_reviews_imbalanced = []
for review in tqdm.tqdm(imdb_text_moderate):
  raw = BeautifulSoup(review)
  html_remove = raw.get_text()
  temp = simple_preprocess(html_remove)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_imbalanced.append(temp_update)

# Preprocessing our unlabeled data
processed_reviews_unlabeled = []
for review in tqdm.tqdm(imdb_unlabeled):
  raw = BeautifulSoup(review)
  html_remove = raw.get_text()
  temp = simple_preprocess(html_remove)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_unlabeled.append(temp_update)

# Defining the final word2vec model parameters
vec_size = 600
window_size = 15
model_architecture = 1
subsample = 1e-2

# Creating the document level representation using the final word2vec model for each review in the different data sets
w2v_model = Word2Vec(sentences = processed_reviews_train, size = vec_size, window = window_size, sg = model_architecture, sample = subsample, seed = 123)

imdb_imbalanced = np.zeros([len(processed_reviews_imbalanced), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_imbalanced))):
  word_list = []
  for word in processed_reviews_imbalanced[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_imbalanced[i] = np.mean(w2v_model.wv[word_list], axis = 0)

imdb_test = np.zeros([len(processed_reviews_test), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_test))):
  word_list = []
  for word in processed_reviews_test[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_test[i] = np.mean(w2v_model.wv[word_list], axis = 0)

imdb_unlabeled = np.zeros([len(processed_reviews_unlabeled), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_unlabeled))):
  word_list = []
  for word in processed_reviews_unlabeled[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_unlabeled[i] = np.mean(w2v_model.wv[word_list], axis = 0)

# Training the final Naive Bayes model on the imbalanced dataset
imdb_nb = GaussianNB()
imdb_nb_fit = imdb_nb.fit(imdb_imbalanced, imdb_y_moderate)

# Training the final Logistic Regression model on the imbalanced dataset
imdb_logreg = LogisticRegression(penalty = "l2", tol = 0.0001, C = 1, max_iter = 500, random_state = 123, solver = 'liblinear')
imdb_logreg_fit = imdb_logreg.fit(imdb_imbalanced, imdb_y_moderate)

# Training the final SVM model on the imbalanced dataset
imdb_svm = SGDClassifier(alpha = 0.03125, random_state = 123)
imdb_svm_fit = imdb_svm.fit(imdb_imbalanced, imdb_y_moderate)

# Training the final Random Forests model on the imbalanced dataset
imdb_rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 17, max_samples = 0.75, random_state = 123)
imdb_rf_fit = imdb_rf.fit(imdb_imbalanced, imdb_y_moderate)

# Using our final models to obtain predictions on the test and compute the F1-score
imdb_nb_preds = imdb_nb_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_preds)

imdb_logreg_preds = imdb_logreg_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_preds)

imdb_svm_preds = imdb_svm_fit.predict(imdb_test)
imdb_svm_f1 = f1_score(imdb_test_y, imdb_svm_preds)

imdb_rf_preds = imdb_rf_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_preds)

# Outputting the F1-score for the final models
print(f"Naive Bayes F1-score: {imdb_nb_f1}")
print(f"Logistic Regression F1-score: {imdb_logreg_f1}")
print(f"SVM F1-score: {imdb_svm_f1}")
print(f"Random Forest F1-score: {imdb_rf_f1}")

In [ ]:
# Random oversampling
imdb_ros = RandomOverSampler(random_state = 123)
imdb_text_resampled, imdb_label_resampled = imdb_ros.fit_resample(imdb_imbalanced, imdb_y_moderate)

# Training the final Naive Bayes model on the balanced dataset
imdb_nb = GaussianNB()
imdb_nb_fit = imdb_nb.fit(imdb_text_resampled, imdb_label_resampled)

# Training the final Logistic Regression model on the balanced dataset
imdb_logreg = LogisticRegression(penalty = "l2", tol = 0.0001, C = 1, max_iter = 500, random_state = 123, solver = 'liblinear')
imdb_logreg_fit = imdb_logreg.fit(imdb_text_resampled, imdb_label_resampled)

# Training the final SVM model on the balanced dataset
imdb_svm = SGDClassifier(alpha = 0.03125, random_state = 123)
imdb_svm_fit = imdb_svm.fit(imdb_text_resampled, imdb_label_resampled)

# Training the final Random Forest model on the balanced dataset
imdb_rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 17, max_samples = 0.75, random_state = 123)
imdb_rf_fit = imdb_rf.fit(imdb_text_resampled, imdb_label_resampled)

# Using our final models to obtain predictions on the test set and compute the F1-score
imdb_nb_preds = imdb_nb_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_preds)

imdb_logreg_preds = imdb_logreg_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_preds)

imdb_svm_preds = imdb_svm_fit.predict(imdb_test)
imdb_svm_f1 = f1_score(imdb_test_y, imdb_svm_preds)

imdb_rf_preds = imdb_rf_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_preds)

# Outputting the F1-score for the final models
print(f"Naive Bayes F1-score: {imdb_nb_f1}")
print(f"Logistic Regression F1-score: {imdb_logreg_f1}")
print(f"SVM F1-score: {imdb_svm_f1}")
print(f"Random Forest F1-score: {imdb_rf_f1}")

In [ ]:
# Defining the hyperparameter configurations
neighbours = 10

# Oversampling using Synthetic Minority Over-sampling Technique (SMOTE)
imdb_smote = SMOTE(k_neighbors = neighbours, random_state = 123)
imdb_text_resampled, imdb_label_resampled = imdb_smote.fit_resample(imdb_imbalanced, imdb_y_moderate)

# Training a Naive Bayes model on the balanced dataset
imdb_nb = GaussianNB()
imdb_nb_fit = imdb_nb.fit(imdb_text_resampled, imdb_label_resampled)

# Training a Logistic Regression model on the balanced dataset
imdb_logreg = LogisticRegression(penalty = "l2", tol = 0.0001, C = 1, max_iter = 500, random_state = 123, solver = 'liblinear')
imdb_logreg_fit = imdb_logreg.fit(imdb_text_resampled, imdb_label_resampled)

# Training an SVM model on the balanced dataset
imdb_svm = SGDClassifier(alpha = 0.03125, random_state = 123)
imdb_svm_fit = imdb_svm.fit(imdb_text_resampled, imdb_label_resampled)

# Training a Random Forest model on the balanced dataset
imdb_rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 17, max_samples = 0.75, random_state = 123)
imdb_rf_fit = imdb_rf.fit(imdb_text_resampled, imdb_label_resampled)

# Using our final models to obtain predictions on the test set and compute the F1-score
imdb_nb_preds = imdb_nb_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_preds)

imdb_logreg_preds = imdb_logreg_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_preds)

imdb_svm_preds = imdb_svm_fit.predict(imdb_test)
imdb_svm_f1 = f1_score(imdb_test_y, imdb_svm_preds)

imdb_rf_preds = imdb_rf_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_preds)

# Outputting the F1-score for the final models
print(f"Naive Bayes F1-score: {imdb_nb_f1}")
print(f"Logistic Regression F1-score: {imdb_logreg_f1}")
print(f"SVM F1-score: {imdb_svm_f1}")
print(f"Random Forest F1-score: {imdb_rf_f1}")

In [ ]:
# Random undersampling
imdb_rus = RandomUnderSampler(random_state = 123)
imdb_text_resampled, imdb_label_resampled = imdb_rus.fit_resample(imdb_imbalanced, imdb_y_moderate)

# Training the final Naive Bayes model on the balanced dataset
imdb_nb = GaussianNB()
imdb_nb_fit = imdb_nb.fit(imdb_text_resampled, imdb_label_resampled)

# Training the final Logistic Regression model on the balanced dataset
imdb_logreg = LogisticRegression(penalty = "l2", tol = 0.0001, C = 1, max_iter = 500, random_state = 123, solver = 'liblinear')
imdb_logreg_fit = imdb_logreg.fit(imdb_text_resampled, imdb_label_resampled)

# Training the final SVM model on the balanced dataset
imdb_svm = SGDClassifier(alpha = 0.03125, random_state = 123)
imdb_svm_fit = imdb_svm.fit(imdb_text_resampled, imdb_label_resampled)

# Training the final Random Forest model on the balanced dataset
imdb_rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 17, max_samples = 0.75, random_state = 123)
imdb_rf_fit = imdb_rf.fit(imdb_text_resampled, imdb_label_resampled)

# Using our final models to obtain predictions on the test set and compute the F1-score
imdb_nb_preds = imdb_nb_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_preds)

imdb_logreg_preds = imdb_logreg_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_preds)

imdb_svm_preds = imdb_svm_fit.predict(imdb_test)
imdb_svm_f1 = f1_score(imdb_test_y, imdb_svm_preds)

imdb_rf_preds = imdb_rf_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_preds)

# Outputting the F1-score for the final models
print(f"Naive Bayes F1-score: {imdb_nb_f1}")
print(f"Logistic Regression F1-score: {imdb_logreg_f1}")
print(f"SVM F1-score: {imdb_svm_f1}")
print(f"Random Forest F1-score: {imdb_rf_f1}")

In [ ]:
# Undersampling using Tomek's Links
imdb_tl = TomekLinks()
imdb_text_resampled, imdb_label_resampled = imdb_tl.fit_resample(imdb_imbalanced, imdb_y_moderate)

# Training the final Naive Bayes model on the balanced dataset
imdb_nb = GaussianNB()
imdb_nb_fit = imdb_nb.fit(imdb_text_resampled, imdb_label_resampled)

# Training the final Logistic Regression model on the balanced dataset
imdb_logreg = LogisticRegression(penalty = "l2", tol = 0.0001, C = 1, max_iter = 500, random_state = 123, solver = 'liblinear')
imdb_logreg_fit = imdb_logreg.fit(imdb_text_resampled, imdb_label_resampled)

# Training the final SVM model on the balanced dataset
imdb_svm = SGDClassifier(alpha = 0.03125, random_state = 123)
imdb_svm_fit = imdb_svm.fit(imdb_text_resampled, imdb_label_resampled)

# Training the final Random Forest model on the balanced dataset
imdb_rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 17, max_samples = 0.75, random_state = 123)
imdb_rf_fit = imdb_rf.fit(imdb_text_resampled, imdb_label_resampled)

# Using our final models to obtain predictions on the test set and compute the F1-score
imdb_nb_preds = imdb_nb_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_preds)

imdb_logreg_preds = imdb_logreg_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_preds)

imdb_svm_preds = imdb_svm_fit.predict(imdb_test)
imdb_svm_f1 = f1_score(imdb_test_y, imdb_svm_preds)

imdb_rf_preds = imdb_rf_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_preds)

# Outputting the F1-score for the final models
print(f"Naive Bayes F1-score: {imdb_nb_f1}")
print(f"Logistic Regression F1-score: {imdb_logreg_f1}")
print(f"SVM F1-score: {imdb_svm_f1}")
print(f"Random Forest F1-score: {imdb_rf_f1}")

In [ ]:
# Preparing our data for augmentation (random substitution)
augment_index = np.array(imdb_text_moderate)[imdb_y_moderate == 0]
augment_index = augment_index.tolist()
augment_index = [''.join(ele) for ele in augment_index]

# Preprocessing our data
processed_reviews_aug = []
for review in tqdm.tqdm(augment_index):
    raw = BeautifulSoup(review)
    html_remove = raw.get_text()
    processed_reviews_aug.append(html_remove)

# Defining the hyperparameter configuration
aug_perc = 0.1
num_augment = 2

# Data augmentation (random subsitution)
aug = naw.SynonymAug(aug_p = aug_perc, stopwords = stopwords_list)
augmented = []
for i in [1, num_augment]:
  for review in augment_index:
    temp = aug.augment(review)
    augmented.append(temp)
    augmented = [''.join(ele) for ele in augmented]

# Preprocessing our augmented data
processed_reviews_aug_post = []
for review in tqdm.tqdm(augmented):
  temp = simple_preprocess(review)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_aug_post.append(temp_update)

# Creating the document level representation using the final word2vec model for each review in the augmented data set
imdb_train_aug = np.zeros([len(processed_reviews_aug_post), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_aug_post))):
  word_list = []
  for word in processed_reviews_aug_post[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_train_aug[i] = np.mean(w2v_model.wv[word_list], axis = 0)

# Combining the imbalanced and augmented data sets
imdb_train_aug_combined = vstack((imdb_imbalanced, imdb_train_aug))
imdb_y_aug = np.zeros(shape=(len(processed_reviews_aug_post), ))
imdb_y_aug_combined = np.concatenate((imdb_y_moderate, imdb_y_aug))

# Training the final Naive Bayes model on the dataset
imdb_nb = GaussianNB()
imdb_nb_fit = imdb_nb.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Training the final Logistic Regression model on the dataset
imdb_logreg = LogisticRegression(penalty = "l2", tol = 0.0001, C = 1, max_iter = 500, random_state = 123, solver = 'liblinear')
imdb_logreg_fit = imdb_logreg.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Training the final SVM model on the dataset
imdb_svm = SGDClassifier(alpha = 0.03125, random_state = 123)
imdb_svm_fit = imdb_svm.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Training the final Random Forests model on the dataset
imdb_rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 17, max_samples = 0.75, random_state = 123)
imdb_rf_fit = imdb_rf.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Using our models to obtain predictions and compute the F1-score
imdb_nb_preds = imdb_nb_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_preds)

imdb_logreg_preds = imdb_logreg_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_preds)

imdb_svm_preds = imdb_svm_fit.predict(imdb_test)
imdb_svm_f1 = f1_score(imdb_test_y, imdb_svm_preds)

imdb_rf_preds = imdb_rf_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_preds)

# Outputting the F1-score for the final models
print(f"Naive Bayes F1-score: {imdb_nb_f1}")
print(f"Logistic Regression F1-score: {imdb_logreg_f1}")
print(f"SVM F1-score: {imdb_svm_f1}")
print(f"Random Forest F1-score: {imdb_rf_f1}")

In [ ]:
# Defining the hyperparameter configuration
aug_perc = 0.2
num_augment = 2

# Data augmentation (random swap)
aug = naw.RandomWordAug(action="swap", aug_p = aug_perc, stopwords = stopwords_list)
augmented = []
for i in [1, num_augment]:
  for review in augment_index:
    temp = aug.augment(review)
    augmented.append(temp)
    augmented = [''.join(ele) for ele in augmented]

# Preprocessing our augmented data
processed_reviews_aug_post = []
for review in tqdm.tqdm(augmented):
  temp = simple_preprocess(review)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_aug_post.append(temp_update)

# Creating the document level representation using the final word2vec model for each review in the augmented data set
imdb_train_aug = np.zeros([len(processed_reviews_aug_post), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_aug_post))):
  word_list = []
  for word in processed_reviews_aug_post[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_train_aug[i] = np.mean(w2v_model.wv[word_list], axis = 0)

# Combining the imbalanced and augmented data sets
imdb_train_aug_combined = vstack((imdb_imbalanced, imdb_train_aug))
imdb_y_aug = np.zeros(shape=(len(processed_reviews_aug_post), ))
imdb_y_aug_combined = np.concatenate((imdb_y_moderate, imdb_y_aug))

# Training the final Naive Bayes model on the dataset
imdb_nb = GaussianNB()
imdb_nb_fit = imdb_nb.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Training the final Logistic Regression model on the dataset
imdb_logreg = LogisticRegression(penalty = "l2", tol = 0.0001, C = 1, max_iter = 500, random_state = 123, solver = 'liblinear')
imdb_logreg_fit = imdb_logreg.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Training the final SVM model on the dataset
imdb_svm = SGDClassifier(alpha = 0.03125, random_state = 123)
imdb_svm_fit = imdb_svm.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Training the final Random Forests model on the dataset
imdb_rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 17, max_samples = 0.75, random_state = 123)
imdb_rf_fit = imdb_rf.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Using our models to obtain predictions and compute the F1-score
imdb_nb_preds = imdb_nb_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_preds)

imdb_logreg_preds = imdb_logreg_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_preds)

imdb_svm_preds = imdb_svm_fit.predict(imdb_test)
imdb_svm_f1 = f1_score(imdb_test_y, imdb_svm_preds)

imdb_rf_preds = imdb_rf_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_preds)

# Outputting the F1-score for the final models
print(f"Naive Bayes F1-score: {imdb_nb_f1}")
print(f"Logistic Regression F1-score: {imdb_logreg_f1}")
print(f"SVM F1-score: {imdb_svm_f1}")
print(f"Random Forest F1-score: {imdb_rf_f1}")

In [ ]:
# Defining the hyperparameter configuration
aug_perc = 0.2
num_augment = 2

# Data augmentation (random swap)
aug = naw.RandomWordAug(action="delete", aug_p = aug_perc, stopwords = stopwords_list)
augmented = []
for i in [1, num_augment]:
  for review in augment_index:
    temp = aug.augment(review)
    augmented.append(temp)
    augmented = [''.join(ele) for ele in augmented]

# Preprocessing our augmented data
processed_reviews_aug_post = []
for review in tqdm.tqdm(augmented):
  temp = simple_preprocess(review)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_aug_post.append(temp_update)

# Creating the document level representation using the final word2vec model for each review in the augmented data set
imdb_train_aug = np.zeros([len(processed_reviews_aug_post), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_aug_post))):
  word_list = []
  for word in processed_reviews_aug_post[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_train_aug[i] = np.mean(w2v_model.wv[word_list], axis = 0)

# Combining the imbalanced and augmented data sets
imdb_train_aug_combined = vstack((imdb_imbalanced, imdb_train_aug))
imdb_y_aug = np.zeros(shape=(len(processed_reviews_aug_post), ))
imdb_y_aug_combined = np.concatenate((imdb_y_moderate, imdb_y_aug))

# Training the final Naive Bayes model on the dataset
imdb_nb = GaussianNB()
imdb_nb_fit = imdb_nb.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Training the final Logistic Regression model on the dataset
imdb_logreg = LogisticRegression(penalty = "l2", tol = 0.0001, C = 1, max_iter = 500, random_state = 123, solver = 'liblinear')
imdb_logreg_fit = imdb_logreg.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Training the final SVM model on the dataset
imdb_svm = SGDClassifier(alpha = 0.03125, random_state = 123)
imdb_svm_fit = imdb_svm.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Training the final Random Forests model on the dataset
imdb_rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 17, max_samples = 0.75, random_state = 123)
imdb_rf_fit = imdb_rf.fit(imdb_train_aug_combined, imdb_y_aug_combined)

# Using our models to obtain predictions and compute the F1-score
imdb_nb_preds = imdb_nb_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_preds)

imdb_logreg_preds = imdb_logreg_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_preds)

imdb_svm_preds = imdb_svm_fit.predict(imdb_test)
imdb_svm_f1 = f1_score(imdb_test_y, imdb_svm_preds)

imdb_rf_preds = imdb_rf_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_preds)

# Outputting the F1-score for the final models
print(f"Naive Bayes F1-score: {imdb_nb_f1}")
print(f"Logistic Regression F1-score: {imdb_logreg_f1}")
print(f"SVM F1-score: {imdb_svm_f1}")
print(f"Random Forest F1-score: {imdb_rf_f1}")

In [ ]:
# Preparing our data
imdb_text_semi = vstack((imdb_imbalanced, imdb_unlabeled))
imdb_y_semi = np.concatenate((imdb_y_moderate, imdb_unlabeled_y))

# Defining the hyperparameter configuration
thresh = 0.99

# Implementing the self-training algorithm using the final Naive Bayes model as the base classifer
imdb_nb = GaussianNB()
imdb_nb_semi = SelfTrainingClassifier(base_estimator = imdb_nb, threshold = thresh)
imdb_nb_semi_fit = imdb_nb_semi.fit(imdb_text_semi, imdb_y_semi)

# Implementing the self-training algorithm using the final Logistic Regression model as the base classifer
imdb_logreg = LogisticRegression(penalty = "l2", tol = 0.0001, C = 1, max_iter = 500, random_state = 123, solver = 'liblinear')
imdb_logreg_semi = SelfTrainingClassifier(base_estimator = imdb_logreg, threshold = thresh)
imdb_logreg_semi_fit = imdb_logreg_semi.fit(imdb_text_semi, imdb_y_semi)

# Implementing the self-training algorithm using the final Random Forests model as the base classifer
imdb_rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 17, max_samples = 0.75, random_state = 123)
imdb_rf_semi = SelfTrainingClassifier(base_estimator = imdb_rf, threshold = thresh)
imdb_rf_semi_fit = imdb_rf_semi.fit(imdb_text_semi, imdb_y_semi)

# Using our models to obtain predictions and compute the F1-score
imdb_nb_semi_preds = imdb_nb_semi_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_semi_preds)

imdb_logreg_semi_preds = imdb_logreg_semi_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_semi_preds)

imdb_rf_semi_preds = imdb_rf_semi_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_semi_preds)

# Outputting the F1-score for the final models
print(f"Naive Bayes F1-score: {imdb_nb_f1}")
print(f"Logistic Regression F1-score: {imdb_logreg_f1}")
print(f"Random Forest F1-score: {imdb_rf_f1}")